# Objective

get raw data, transform it, and save it into another csv

In [3]:
import numpy as np
import pandas as pd
from pathlib import Path

import cratutils as cu

In [72]:

import importlib
importlib.reload(cu)

<module 'cratutils' from '/Users/yx/Documents/pro/crat/cratutils.py'>

In [20]:
cu.WriteRawDataToCsv()

100%|██████████| 677/677 [00:03<00:00, 198.92it/s]


In [73]:
videos = cu.GetVideos()

for video in videos:
    for frame in video:
        pass

trainVideos, testVideos = cu.SplitVideos(videos)
X_train, y_train = cu.GetXYFromVideos(trainVideos)
X_test, y_test = cu.GetXYFromVideos(testVideos)

In [136]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, GRU, LSTM, Dense, Masking, Attention, MultiHeadAttention, Input, Lambda, Layer, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard

In [130]:
class CustomAttention(Layer):
    
    def __init__(self, return_sequences=True):
        self.return_sequences = return_sequences
        super(CustomAttention,self).__init__()
        
    def build(self, input_shape):
        
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),
                               initializer="zeros")
        
        super(CustomAttention,self).build(input_shape)
        
    def call(self, x):
        
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        output = x*a
        
        if self.return_sequences:
            return output
        
        return K.sum(output, axis=1)

In [139]:

# LSTM

maxlen = max(len(seq) for seq in X_train)

padding_value = 9999

X_train_padded = pad_sequences(X_train, maxlen=maxlen, padding='post', value=padding_value)

num_classes = len(cu.thirdClasses)
num_features = len(X_train[0][0])

y_train_categorical = to_categorical(y_train, num_classes)

model = Sequential()

model.add(Masking(mask_value=padding_value, input_shape=(maxlen, num_features)))

# model.add(Bidirectional(LSTM(units=32, return_sequences=True)))

# model.add(CustomAttention(return_sequences=True))

# model.add(LSTM(32))

from keras_self_attention import SeqSelfAttention

model.add(LSTM(32, return_sequences=True))
model.add(SeqSelfAttention(
        attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
        attention_activation='softmax'))
model.add(LSTM(32))

model.add(Dense(32, activation='relu'))

model.add(Dense(num_classes, activation='linear'))

model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True) , metrics=['accuracy'])

model.summary()

tf_callback = TensorBoard(log_dir='./tensorboard')

model.fit(X_train_padded, y_train_categorical, epochs=50, batch_size=32, callbacks=[tf_callback])

/opt/miniconda3/envs/tf/lib/python3.10/site-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_21"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking_21 (Masking)            │ (None, 65, 13)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_21 (LSTM)                  │ (None, 65, 32)         │         5,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ seq_self_attention              │ (None, 65, 32)         │         1,025 │
│ (SeqSelfAttention)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_22 (LSTM)                  │ (None, 32)             │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 19)             │           627 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,916 (66.08 KB)

 Trainable params: 16,916 (66.08 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
207/207 ━━━━━━━━━━━━━━━━━━━━ 16s 59ms/step - accuracy: 0.1198 - loss: 2.7894
Epoch 2/50
207/207 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - accuracy: 0.1589 - loss: 2.5639
Epoch 3/50
207/207 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - accuracy: 0.2067 - loss: 2.4904
Epoch 4/50
207/207 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - accuracy: 0.2010 - loss: 2.4674
Epoch 5/50
207/207 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - accuracy: 0.2078 - loss: 2.4383
Epoch 6/50
207/207 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - accuracy: 0.2094 - loss: 2.4242
Epoch 7/50
207/207 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - accuracy: 0.2097 - loss: 2.4061
Epoch 8/50
207/207 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - accuracy: 0.2085 - loss: 2.4203
Epoch 9/50
207/207 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - accuracy: 0.2135 - loss: 2.3970
Epoch 10/50
207/207 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - accuracy: 0.2078 - loss: 2.3971
Epoch 11/50
207/207 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - accuracy: 0.2112 - loss: 2.3804
Epoch 12/50
207/207 ━━━━━━━━━━━━━━━━━━━━

In [133]:

# Simple RNN

maxlen = max(len(seq) for seq in X_train)

padding_value = 9999

X_train_padded = pad_sequences(X_train, maxlen=maxlen, padding='post', value=padding_value)

num_classes = len(cu.thirdClasses)
y_train_categorical = to_categorical(y_train, num_classes)

model = Sequential([
    Masking(mask_value=padding_value, input_shape=(maxlen, len(X_train[0][0]))),
    Bidirectional(GRU(units=32, return_sequences=True)),
    Attention(),
    GRU(32),
    Dense(num_classes, activation='softmax'),
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

model.fit(X_train_padded, y_train_categorical, epochs=5, batch_size=32)


/opt/miniconda3/envs/tf/lib/python3.10/site-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'attention_10' (of type Attention) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking_17 (Masking)            │ (None, 65, 13)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_9 (Bidirectional) │ (None, 65, 64)         │         9,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention_10 (Attention)        │ (None, 65, 64)         │           129 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_5 (GRU)                     │ (None, 32)             │         9,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 19)             │           627 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,188 (74.95 KB)

 Trainable params: 19,188 (74.95 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
207/207 ━━━━━━━━━━━━━━━━━━━━ 27s 120ms/step - accuracy: 0.1356 - loss: 2.7680
Epoch 2/5
207/207 ━━━━━━━━━━━━━━━━━━━━ 22s 108ms/step - accuracy: 0.1424 - loss: 2.5623
Epoch 3/5
207/207 ━━━━━━━━━━━━━━━━━━━━ 22s 109ms/step - accuracy: 0.1641 - loss: 2.5541
Epoch 4/5
207/207 ━━━━━━━━━━━━━━━━━━━━ 23s 110ms/step - accuracy: 0.1790 - loss: 2.5228
Epoch 5/5
207/207 ━━━━━━━━━━━━━━━━━━━━ 23s 112ms/step - accuracy: 0.1951 - loss: 2.5137


In [112]:

# Simple RNN

maxlen = max(len(seq) for seq in X_train)

padding_value = 9999

X_train_padded = pad_sequences(X_train, maxlen=maxlen, padding='post', value=padding_value)

num_classes = len(cu.thirdClasses)
y_train_categorical = to_categorical(y_train, num_classes)

model = Sequential([
    GRU(units=64, input_shape=(maxlen, len(X_train[0][0]))),
    Masking(mask_value=padding_value, input_shape=(maxlen, len(X_train[0][0]))),
    Dense(num_classes, activation='softmax'),
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

model.fit(X_train_padded, y_train_categorical, epochs=5, batch_size=32)

/opt/miniconda3/envs/tf/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/opt/miniconda3/envs/tf/lib/python3.10/site-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, 64)             │        15,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ masking_5 (Masking)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 19)             │         1,235 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,403 (64.07 KB)

 Trainable params: 16,403 (64.07 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
207/207 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - accuracy: 0.1136 - loss: 2.9508
Epoch 2/5
207/207 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.1483 - loss: 2.5854
Epoch 3/5
207/207 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.1472 - loss: 2.5613
Epoch 4/5
207/207 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.1378 - loss: 2.5661
Epoch 5/5
207/207 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.1429 - loss: 2.5735


In [114]:
X_test_padded = pad_sequences(X_test, maxlen=maxlen, padding='post', dtype='float32', value=padding_value)

# Convert test labels to categorical (for multi-class classification)
y_test_categorical = to_categorical(y_test, num_classes)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test_padded, y_test_categorical)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Predicting labels for the test set
y_pred = model.predict(X_test_padded)

# To get the predicted classes (argmax across softmax probabilities)
y_pred_classes = np.argmax(y_pred, axis=1)

print(f"Predicted Classes: {y_pred_classes}")
print(f"True Labels: {y_test}")

50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.1895 - loss: 2.5813
Test Loss: 2.615651845932007
Test Accuracy: 0.16040737926959991
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
Predicted Classes: [5 5 5 ... 5 5 5]
True Labels: [1 1 1 ... 5 5 5]


In [100]:
videos['0.csv'][0].columns

Index(['TIMESTAMP', 'TRACK_ID', 'OBJECT_TYPE', 'X', 'Y', 'V_X', 'V_Y', 'A_X',
       'A_Y', 'YAW', 'DYAW', 'DDYAW', 'first_class', 'second_class',
       'third_class', 'fileName'],
      dtype='object')